In [12]:
import sys
import nltk
import sklearn
import pandas
import numpy

import pandas as pd
import numpy as np
df = pd.read_csv('Bug Reports Dataset.csv')
print (df.head(5))

   Bug ID Product Component          Assignee    Status Classification  \
0  528905     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
1  530231     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
2  530758     JDT      Core          jarthana  VERIFIED        Eclipse   
3  531990     JDT      Core    jdt-core-inbox  VERIFIED        Eclipse   
4  532137     JDT      Core  register.eclipse  VERIFIED        Eclipse   

  Priority            Opened  \
0       P1  12/18/2017 11:46   
1       P1    1/24/2018 5:16   
2       P1    2/5/2018 20:38   
3       P1     3/5/2018 0:49   
4       P1    3/7/2018 16:17   

                                             Summary       Category  
0  JDT UI Gerrit failing with "invalid location f...  Configuration  
1  Compilation failure in M20180123-0400 in jdt.c...  Configuration  
2                    Build failure in I20180205-2000  Configuration  
3                    Build failure in I20180304-2000  Configuration  
4  Todays up

In [13]:
classes = df['Priority']
print(classes.value_counts())

P3    925
P2    539
P1    294
P5    225
P4    155
Name: Priority, dtype: int64


In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform (classes)
print(classes[:8])
print(Y[:8])
priority = df['Priority']
print(priority[:10])

0    P1
1    P1
2    P1
3    P1
4    P1
5    P1
6    P1
7    P1
Name: Priority, dtype: object
[0 0 0 0 0 0 0 0]
0    P1
1    P1
2    P1
3    P1
4    P1
5    P1
6    P1
7    P1
8    P1
9    P1
Name: Priority, dtype: object


In [15]:
# Categorical boolean mask
categorical_feature_mask = df.dtypes == object

In [16]:
categorical_feature_mask

Bug ID            False
Product            True
Component          True
Assignee           True
Status             True
Classification     True
Priority           True
Opened             True
Summary            True
Category           True
dtype: bool

In [17]:
# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()
categorical_cols

['Product',
 'Component',
 'Assignee',
 'Status',
 'Classification',
 'Priority',
 'Opened',
 'Summary',
 'Category']

In [18]:
pri_col = []
for i in categorical_cols:
    if i != 'Summary':
        pri_col.append(i)
print(pri_col)

['Product', 'Component', 'Assignee', 'Status', 'Classification', 'Priority', 'Opened', 'Category']


In [19]:
le = LabelEncoder()
df[pri_col] = df[pri_col].apply(lambda col: le.fit_transform(col))
df[pri_col].head(10)

,Product,Component,Assignee,Status,Classification,Priority,Opened,Category
0,44,41,144,4,4,0,475,0
1,44,41,144,4,4,0,82,0
2,44,41,141,6,4,0,717,0
3,44,41,144,6,4,0,931,0
4,44,41,273,6,4,0,948,0
5,44,41,141,6,4,0,1367,0
6,22,41,7,1,8,0,105,0
7,14,272,227,2,9,0,1407,0
8,14,98,227,5,9,0,149,0
9,14,116,227,2,9,0,1610,0


In [20]:
df.Priority.value_counts()

2    925
1    539
0    294
4    225
3    155
Name: Priority, dtype: int64

In [35]:
X  = df[['Product','Classification','Component', 'Category', 'Assignee','Status']]

In [36]:
print(X.shape)

(2138, 6)


In [37]:
Y = df['Priority']

In [38]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [40]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [41]:
predictions = text_classifier.predict(X_test)

In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[ 38  11   8   0   2]
 [ 10  99  12   0   0]
 [  7  12 151   8   4]
 [  1   1  10  15   2]
 [  3   2   4   0  28]]
              precision    recall  f1-score   support

           0       0.64      0.64      0.64        59
           1       0.79      0.82      0.80       121
           2       0.82      0.83      0.82       182
           3       0.65      0.52      0.58        29
           4       0.78      0.76      0.77        37

    accuracy                           0.77       428
   macro avg       0.74      0.71      0.72       428
weighted avg       0.77      0.77      0.77       428

0.7733644859813084


In [51]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [52]:
print(confusion_matrix(y_test,y_pred))  
print(metrics.precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print(metrics.recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print(metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print(metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred,zero_division=1))  

[[ 11  22  26   0   0]
 [  5  95  21   0   0]
 [  7  13 146   0  16]
 [  1   9  17   0   2]
 [  4   4  16   0  13]]
0.5931191405623633
0.6641604010025063
0.6175559758083512
0.6191588785046729
              precision    recall  f1-score   support

           0       0.39      0.19      0.25        59
           1       0.66      0.79      0.72       121
           2       0.65      0.80      0.72       182
           3       1.00      0.00      0.00        29
           4       0.42      0.35      0.38        37

    accuracy                           0.62       428
   macro avg       0.62      0.43      0.41       428
weighted avg       0.62      0.62      0.58       428



In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [54]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred_test))  
print(metrics.precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print(metrics.recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print(metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
print(metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred_test,zero_division=1))  

[[  1  25  33   0   0]
 [  0  81  40   0   0]
 [  5  21 155   0   1]
 [  0   9  20   0   0]
 [  0   5  28   0   4]]
0.5931191405623633
0.6641604010025063
0.6175559758083512
0.6191588785046729
              precision    recall  f1-score   support

           0       0.17      0.02      0.03        59
           1       0.57      0.67      0.62       121
           2       0.56      0.85      0.68       182
           3       1.00      0.00      0.00        29
           4       0.80      0.11      0.19        37

    accuracy                           0.56       428
   macro avg       0.62      0.33      0.30       428
weighted avg       0.56      0.56      0.48       428



In [55]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(criterion = 'entropy').fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
print("Accuracy:",metrics.accuracy_score(y_test, dtree_predictions))

Accuracy: 0.7336448598130841


In [56]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,dtree_predictions))  
print(classification_report(y_test,dtree_predictions))  
print(accuracy_score(y_test, dtree_predictions))

[[ 40  11   6   0   2]
 [ 16  94  10   1   0]
 [  9  13 141   7  12]
 [  1   1  11  14   2]
 [  3   2   6   1  25]]
              precision    recall  f1-score   support

           0       0.58      0.68      0.63        59
           1       0.78      0.78      0.78       121
           2       0.81      0.77      0.79       182
           3       0.61      0.48      0.54        29
           4       0.61      0.68      0.64        37

    accuracy                           0.73       428
   macro avg       0.68      0.68      0.67       428
weighted avg       0.74      0.73      0.73       428

0.7336448598130841
